In [1]:
#imporing the necessary libraries for our analysis
import numpy as np 
import pandas as pd 
import json
#this is to help get the fie directory for our dataset
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/imdb-review-dataset/part-05.json
/kaggle/input/imdb-review-dataset/part-01.json
/kaggle/input/imdb-review-dataset/part-02.json
/kaggle/input/imdb-review-dataset/part-06.json
/kaggle/input/imdb-review-dataset/sample.json
/kaggle/input/imdb-review-dataset/part-03.json
/kaggle/input/imdb-review-dataset/part-04.json


In [2]:
#importing our dataset
movies = "/kaggle/input/imdb-review-dataset/" 
files = ["part-01.json", "part-02.json", "part-03.json", "part-04.json", "part-05.json", "part-06.json"]
listing = list()


for i in files:
  #reading the files
  with open(f"{movies}/{i}", mode='r') as file:
    new_reviews = json.load(file)
    #we will use extend() method as it adds all the elements of an iterable list/tuple to the end of the list.
    listing.extend(new_reviews)
            
# creating a dataframe called movie_df from the list above
movie_df = pd.DataFrame(listing)

In [13]:
movie_df.head(3)

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,rw5704482,raeldor-96879,After Life (2019– ),9,Very Strong Season 2,3 May 2020,0,"I enjoyed the first season, but I must say I t...","[1, 1]"
1,rw5704483,dosleeb,The Valhalla Murders (2019– ),6,Icelandic detectives?,3 May 2020,0,I know Iceland is a small country and police d...,"[2, 2]"
2,rw5704484,brightconscious,Special OPS (2020– ),7,Nothing special,3 May 2020,0,"Except K K , no other actor looks comfortable ...","[0, 0]"


**Understanding the data****

In [14]:
#checking the shape of the data
movie_df.shape

(5571499, 9)

The dataset has 5,571,499 rows and 9 columns

In [16]:
#checking the datatypes of the columns
movie_df.dtypes

review_id         object
reviewer          object
movie             object
rating            object
review_summary    object
review_date       object
spoiler_tag        int64
review_detail     object
helpful           object
dtype: object

convert the rating column to integer

In [15]:
#checking the missing values
movie_df.isnull().sum()

review_id              0
reviewer               0
movie                  0
rating            662849
review_summary         0
review_date            0
spoiler_tag            0
review_detail          0
helpful                0
dtype: int64

In [ ]:
# Count the entries for the reviews
reviews_count = len(listing)
print(f"Loading complete, {reviews_count:,} items in reviews")

In [19]:
#checking the % of the missing values
rating_missing = 662849
perc_missing = ((rating_missing/reviews_count)*100)
print(perc_missing)

11.897139351546146


There are 662,849 missing values in the ratings column which is about 12% of the total entries. We can drop the missing values

**Data Cleaning**

In [22]:
#dropping the null values
movie_df.dropna(inplace=True)

In [5]:
# Selecting the columns we need for our analysis and create a new dataframe; movie1_df
movie1_df = movie_df.loc[:, ["reviewer", "movie", "rating", "review_summary", "review_detail"]]



In [23]:
# Reviewing the top rows for our movie1_df
movie1_df.head(5)

,reviewer,movie,rating,review_summary,review_detail
2639616,moondust17,The Punisher (2004),10.0,Great Movie,This was a great movie. I love movies when th...
2432038,ShadeGrenade,I Didn't Know You Cared (1975–1979),10.0,Its grim up North!,'I Didn't Know You Cared' crept unheralded int...
2432033,Crossfire3636,Inception (2010),10.0,Can a movie get any better than being an exper...,Inception was one of those movies after which ...
2432031,DonnieDarko6666,Srpski film (2010),10.0,"just a few thoughts on the movie ""A Serbian Film""","I've seen "" A Serbian Film"" twice now in betwe..."
2432030,bosu_rares,Adela (1985),10.0,A Beautiful Romance,"I know it sounds simple, but the word fully de..."


In [7]:
# converting the datatype for the ratings column to numeric 
movie1_df["rating"] = pd.to_numeric(movie1_df["rating"])

# our target column is ratings so we sort the df using the descending order
movie1_df.sort_values(by='rating', ascending=False, inplace=True)

movie1_df.head()

,reviewer,movie,rating,review_summary,review_detail
2639616,moondust17,The Punisher (2004),10.0,Great Movie,This was a great movie. I love movies when th...
2432038,ShadeGrenade,I Didn't Know You Cared (1975–1979),10.0,Its grim up North!,'I Didn't Know You Cared' crept unheralded int...
2432033,Crossfire3636,Inception (2010),10.0,Can a movie get any better than being an exper...,Inception was one of those movies after which ...
2432031,DonnieDarko6666,Srpski film (2010),10.0,"just a few thoughts on the movie ""A Serbian Film""","I've seen "" A Serbian Film"" twice now in betwe..."
2432030,bosu_rares,Adela (1985),10.0,A Beautiful Romance,"I know it sounds simple, but the word fully de..."


The highest rating given is 10.We will choose the movie with the most number of ratings based with how many times rating 10 has occured.

**Question1**

In [24]:
#creating a dataframe which consists of the movie and ratings column
# Acolumn named ratings occurences is created
grouping_df = movie1_df.groupby(['movie', 'rating']).size().reset_index(name='ratings_occurences')
grouping_df.sort_values(by=['rating','ratings_occurences'],ascending=[False,False],inplace=True)

grouping_df.head()

,movie,rating,ratings_occurences
225159,Dil Bechara (2020),10.0,7188
890288,The Shawshank Redemption (1994),10.0,5331
77324,Avengers: Endgame (2019),10.0,4363
1029597,小丑 (2019),10.0,4145
535809,Mrs. Serial Killer (2020),10.0,3978


In [25]:
#the dopest movie is the one with the highest rating occurences
dope_movie = grouping_df.nlargest(1,'ratings_occurences')['movie'].values[0]
print('The movie with the highest rating occurences is:',dope_movie)

The movie with the highest rating occurences is: Dil Bechara (2020)


**Question 2******

In [26]:
#2. How many reviewers gave it the highest rating

highest_rating = grouping_df.loc[grouping_df['movie'] == dope_movie, 'rating'].max()
#total number of individual reviewers who gave the dope movie the highest ratings
individual_count = movie1_df[(movie1_df['movie'] == dope_movie) & (movie1_df['rating'] == highest_rating)].reviewer.nunique()
print(f"{individual_count} individual reviewers contributed to the highest rating of the dopest movie")

7132 individual reviewers contributed to the highest rating of the dopest movie


**Question 3**

In [12]:
#What’s its least rating and review detail?
least_rating = movie1_df.loc[movie1_df['movie'] == dope_movie, 'rating'].min()
least_rated = movie1_df[(movie1_df['movie'] == dope_movie) & (movie1_df['rating'] == least_rating)]
least_rated = least_rated.tail(1)
rating = least_rated['rating'].values[0]
review_detail = least_rated['review_detail'].values[0]
print("Rating: ", rating)
print("Review Detail: ", review_detail)

Rating:  1.0
Review Detail:  The movie seems kinda rushed and scenes seem rushed too. Terrible copy of TFIOS. Direction was terrible. Everything about this movie wqs terrible
